In [10]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [11]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [12]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [13]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [14]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [15]:
def getStakeAndRewards():
    stakes = dict()
    rewards = dict()
    shards = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            rewards[address] = reward_accumulated
            by_shard_metrics = i['metrics']['by-bls-key']
            v_stakes = dict()
            v_shards = dict()
            for by_shard_metric in by_shard_metrics:
                bls_key = by_shard_metric['key']['bls-public-key']
                e_stake = float(by_shard_metric['key']['effective-stake'])
                shard_id = by_shard_metric['key']['shard-id']
                v_stakes[bls_key] = e_stake
                v_shards[bls_key] = shard_id
            stakes[address] = v_stakes
            shards[address] = v_shards
    return rewards, stakes, shards

In [16]:
def getRewards():
    rewards = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            rewards[address] = reward_accumulated
    return rewards

In [17]:
def proportional(l1, l2):
    return l1 == l2

In [18]:
def extract(lst): 
    return [item[0] for item in lst] 

In [22]:
def check(lst1, lst2):
    keys1 = [item[0] for item in lst1]
    keys2 = [item[0] for item in lst2]
    stakes = [item[1] for item in lst1]
    rewards = [item[1] for item in lst2]
    l = len(keys1)
    i = 0
    j = 0
    while i < l:
        if keys1[i] == keys1[i]:
            i = i + 1
        else:
            stake = stakes[i]
            reward = rewards[i]
            i1 = i
            i2 = i
            j = i
            while stakes[i1] == stake:
                i1 = i1 + 1
            while rewards[i2] == reward:
                i2 = i2 + 1
            if i1 != i2:
                return False
            while i < i1:
                k = j
                found = False
                while k < i2:
                    if keys1[i] == keys2[k]:
                        found = True
                        break
                    k = k + 1
                if found == False:
                    return False
                i = i + 1
            i = i1
     
    return True   

In [23]:
def R3_test():
    print("Test-R3: High stakers earn more reward than low stakers")
    num = int(input("enter the number of blocks you want to test: "))
    
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    while block < last_block:
        block = getBlockNumber()
        
    new_epoch_block = block + 2 # first block is problematic, hence going for second.
    while block < new_epoch_block:
        block = getBlockNumber()
    
    print("new epoch first block reached", new_epoch_block, "will begin testing")
    new_epoch_block = block + 1
    
    rewards, stakes, shards = getStakeAndRewards()
    
    print("obtained first block stakes and rewards", "total stakes found=", \
          len(stakes), "total rewards found=", len(rewards))
    
    while block < new_epoch_block:
        block = getBlockNumber()
        
    print("new epoch second block reached", block, "will begin comparing stakes and rewards")  

    iterations = 0
    while iterations < num:
        new_rewards = getRewards()
        last_block = block
        current_block = last_block
        
        key_to_stake = dict()
        key_to_reward = dict()
        key_to_shard = dict()
        for addr, reward in new_rewards.items():
            if addr in rewards and addr in stakes and addr in shards:
                addr_reward = reward - rewards[addr]
                slots = len(stakes[addr])
                per_slot_reward = addr_reward / slots
                for key, stake in stakes[addr].items():
                    key_to_reward[key] = per_slot_reward
                    if key in stakes[addr]:
                        key_to_stake[key] = stakes[addr][key]
                    if key in shards[addr]:
                        key_to_shard[key] = shards[addr][key]
        
        shard_rewards = dict()
        shard_stakes = dict()
        for key, shard in key_to_shard.items():
            if shard not in shard_stakes:
                shard_stakes[shard] = dict()
            if shard not in shard_rewards:
                shard_rewards[shard] = dict()
            shard_stakes[shard][key] = key_to_stake[key]
            shard_rewards[shard][key] = key_to_reward[key]
            
        for shard in shard_rewards.keys():
            sorted_stakes = sorted(shard_stakes[shard].items(), key=lambda kv: kv[1], reverse = True)
            sorted_rewards = sorted(shard_rewards[shard].items(), key=lambda kv: kv[1], reverse = True)
            stake_keys = extract(sorted_stakes)
            reward_keys = extract(sorted_rewards)
            print("comparison to begin, two lengths:", len(stake_keys), len(reward_keys))
            if check(sorted_stakes, sorted_rewards) == False:
                print("Test-R3: on shard", shard, "failed")
                print("validators sorted by stakes: ", stake_keys)
                print("validators sorted by reward: ", reward_keys)
            else:
                print("Test-R3: on shard: ", shard, "succeeded")
            
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        rewards = new_rewards
        iterations = iterations + 1
        if current_block > last_block:
            break

In [24]:
R3_test()

Test-R3: High stakers earn more reward than low stakers
enter the number of blocks you want to test: 1
current and last block numbers 12273 12273
new epoch first block reached 12275 will begin testing
obtained first block stakes and rewards total stakes found= 112 total rewards found= 112
new epoch second block reached 12276 will begin comparing stakes and rewards
comparison to begin, two lengths: 64 64
Test-R3: on shard:  0 succeeded
comparison to begin, two lengths: 45 45
Test-R3: on shard:  1 succeeded
comparison to begin, two lengths: 43 43
Test-R3: on shard:  2 succeeded
comparison to begin, two lengths: 40 40
Test-R3: on shard:  3 succeeded
